### Create xcom directory

In [ ]:
! mkdir -p /airflow/xcom

### Imports

In [ ]:
import great_expectations as ge
import requests
import json
from pprint import pprint as pp
from io import StringIO

### Fetch styrk koder and load into ge dataframe

In [ ]:
res = requests.get("https://data.adeo.no/api/nav-opendata/styrk-koder/styrk_s3.csv")

In [ ]:
buffer = StringIO(res.text)

In [ ]:
ge_df = ge.read_csv(buffer)

In [ ]:
ge_df.head()

### Run validation tests

In [ ]:
validation_errors = []

In [ ]:
# Range check 1-4 for level column
valid_range = list(range(1,5))
res = ge_df.expect_column_values_to_be_in_set('level', valid_range)
validation_errors.append(res) if not res["success"] else print("Successful range check 1-4 for level columnRange check 1-4 for level column")

In [ ]:
# Regex check for name column
valid_regex = "[\wæøåÆØÅ]"
res = ge_df.expect_column_values_to_match_regex('name', regex=valid_regex)
validation_errors.append(res) if not res["success"] else print("Successful regex check for name column")

In [ ]:
# Check for valid max/min styrk code column
styrk_min = 0
styrk_max = 9000
res = ge_df.expect_column_values_to_be_between('code', min_value=styrk_min, max_value=styrk_max)
validation_errors.append(res) if not res["success"] else print("Successful check for valid max/min styrk code column")

### Print validation errors to std

In [ ]:
print("Validation errors")
pp(validation_errors)

### Write validation errors to xcom output file

In [ ]:
with open("/airflow/xcom/return.json", "w") as f:
    f.write(json.dumps(validation_errors))